# Library Import

In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset

In [2]:
from KoBERT.kobert.utils import get_tokenizer
from KoBERT.kobert.pytorch_kobert import get_pytorch_kobert_model

In [3]:
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

In [4]:
import gluonnlp as nlp

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

from tqdm import tqdm, tqdm_notebook
import sys

# Main

In [7]:
%matplotlib inline

In [8]:
plt.style.use('seaborn')
fm.get_fontconfig_fonts()
font_location = '/content/drive/MyDrive/Colab Notebooks/BERT_BMC/font/NanumGothic.ttf'
fprop = fm.FontProperties(fname=font_location)

In [9]:
import random
random.seed(1004)

In [10]:
num_class = 5

In [11]:
device = torch.device("cuda:0")

# import torch_xla
# import torch_xla.core.xla_model as xm

# net = torchvision.models.alexnet(num_classes=num_class)

# # Acquires the default Cloud TPU core and moves the model to it
# device = xm.xla_device()
# net = net.to(device)

In [12]:
bertmodel, vocab = get_pytorch_kobert_model()
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model
using cached model
using cached model


In [14]:
# 데이터 로딩
df = pd.read_csv("data2/BMC_data.csv")
# df

In [16]:
df2 = pd.read_csv("data2/고객세그먼트.csv")
# df2




In [17]:
total_df = pd.DataFrame()

In [18]:
total_df['기업명'] = df['기업명']

In [19]:
# text컬럼에 모든 컬럼 텍스트 합치기
total_df['text'] = "고객 세그먼트는 "+ df['고객 세그먼트'] + "." +\
            " 가치제안은 " + df['가치제안'] + "." +\
            " 마케팅채널은 " + df['마케팅 채널'] + "." +\
            " 고객관계는 " + df['고객관계'] + "." +\
            " 수익원은 " + df['수익원'] + "." +\
            " 핵심활동은 " + df['핵심활동'] + "." +\
            " 핵심자원은 " + df['핵심자원'] + "." +\
            " 핵심파트너는 " + df['핵심파트너'] + "."\
            " 비용구조는 " + df['비용구조'] + "."

In [20]:
# a = "고객 세그먼트는 "+ df['고객 세그먼트'] + " 가치제안은 " + df['가치제안'] + " 마케팅채널은 " + df['마케팅 채널'] + " 고객관계는 " +\
#     df['고객관계'] + " 수익원은 " + df['수익원'] + " 핵심활동은 " + df['핵심활동'] + " 핵심자원은 " + df['핵심자원'] + " 핵심파트너는 " +\
#     df['핵심파트너'] + " 비용구조는 " + df['비용구조']

In [21]:
total_df

,기업명,text
0,에이앤유디자인그룹건축사사무소,"고객 세그먼트는 현대건설, 지에스건설 등 건설사\nLH공사, SH공사, 지방 도시공..."
1,멜텍스코리아 주식회사,고객 세그먼트는 전자부품MLCC(Multi Layer Ceramic Condense...
2,에스티엑스건설,고객 세그먼트는 주거용 건물 건축주(입주자아파트 등)\n건설부문 외주 및 자재 생산...
3,충청산업보건연구원(주),"고객 세그먼트는 관공서(행정기관)석면사업부(조사, 감리, 측정)\n제조업(대기업)작..."
4,대동산업(주),"고객 세그먼트는 건설사, 시행사, 설계사무소\n도소매 유통업체\n건축주\nLH공사,..."
...,...,...
499,㈜케이바스,"고객 세그먼트는 욕실 용품 유통업체\n시행사, 시공사, 중소 건설사. 가치제안은 소..."
500,진우엔티,"고객 세그먼트는 노즐 및 밸브를 사용하는 기업\n분수, 아파트 정원 공사업체\n환경..."
501,㈜에어붐,고객 세그먼트는 당사 1단계 목표 : 초중고등학교 \n2단계 목표 : B to C...
502,㈜킴스코파쎄,"고객 세그먼트는 건설사(민영, 공영), 설계사무소, 시행사\n건축자재 도소매 업체,..."


# Functions

In [22]:
# 데이터셋 컬럼의 중복을 제외한 2음절 이상의 vocab에 등록된 단어를 list로 만드는 함수 
# 인자로 컬럼 하나의 데이터프레임이 들어가야함. e.g.) df['고객세그먼트'] 

def store_tokenized_words(one_dataset):
    # 총 tokenizing 단어 리스트에 저장
    token_list = []
    for i in range(len(df)):
#         print(tok(one_dataset[i]))
        token_list.append(tok(one_dataset[i]))

    # set에 tokenizing 단어 저장(중복제거)
    token_set = []
    for i in range(len(token_list)):
        for j in range(len(token_list[i])):
            token_set.append(token_list[i][j])
    # set(token_list)

    unique_set = set(token_set)
    unique_list = list(unique_set)

    # 한글자 tokenizing 단어 제외한 후 unique_company_list에 저장
    unique_company_list = []
    for i in unique_list:
        if (i[0] == '▁') & (len(i) >=3):
            unique_company_list.append(i)
    #         print(i)  
        if (i[0] != '▁') & (len(i) >=2):
            unique_company_list.append(i)
    #         print(i)

    # 리스트의 단어에서 '_' 제거
    for i in range(len(unique_company_list)):
        unique_company_list[i] = unique_company_list[i].replace('▁',"")

#     print(unique_company_list)
    return unique_company_list

In [23]:
# 컬럼 내 모든 어절을 word_set에 저장(중복제외)하는 함수
# 인자로 컬럼 하나의 데이터프레임이 들어가야함. e.g.) df['고객세그먼트'] 

def store_segmented_words(one_dataset):
    word_set = set()
    rm_list = ["(", ")", "\n", '"', ","]

    for sentence in one_dataset:
        for rm in rm_list:
            sentence = sentence.replace(rm, " ")

        sentence = sentence.split(" ")
        for word in sentence:
#             print(word)
            word_set.add(word)

    return word_set

In [24]:
# vocab 단어와 컬럼의 어절을 비교하여 vocab에 저장되지 않은 단어를 리스트로 추출하는 함수

def store_untokenized_words(segmented_words, tokenized_words):
    modifying_list = []

    for word in segmented_words:
    #     print(word)
        if word not in tokenized_words:
            modifying_list.append(word)

    return modifying_list

In [27]:
tokenized_words = store_tokenized_words(total_df['text'])
len(tokenized_words)

3222

In [28]:
segmented_words = store_segmented_words(total_df['text'])
len(segmented_words)

22793

In [29]:
untokenized_words = store_untokenized_words(segmented_words, tokenized_words)
len(untokenized_words)

20853

In [30]:
# untokenized_words

In [31]:
# untokenized_words_df 데이터프레임 변환
untokenized_words_df = pd.DataFrame(untokenized_words, columns=["untokenized_word"])

In [34]:
# csv로 저장
untokenized_words_df.to_csv("data2/untokenized_words_df.csv", encoding='utf-8-sig')

In [35]:
tok('SAMSUNG')

['▁S', 'A', 'MS', 'U', 'N', 'G']

In [36]:
tok('삼성')

['▁삼성']

In [37]:
tok('지에스')

['▁지', '에', '스']

In [38]:
tok('GS')

['▁GS']

In [39]:
tok('SNS')

['▁SNS']

In [40]:
tok('facebook')

['▁', 'f', 'ac', 'e', 'b', 'o', 'o', 'k']

In [41]:
tok('global')

['▁', 'g', 'lo', 'b', 'al']

In [42]:
modifying_list = []
for token in unique_list:
    if token not in unique_company_list:
        modifying_list.append(token)
        

NameError: name 'unique_list' is not defined

In [44]:
len(modifying_list)

0

In [ ]:
# modifying_list

In [45]:
import re

In [46]:
p = re.compile("[a-z]+")

In [47]:
m = p.match("python")
print(m)

<re.Match object; span=(0, 6), match='python'>


In [48]:
m = p.match("3 python")
print(m)

None


In [49]:
p = re.compile('[a-z]+')
m = p.match('string goes here')
if m:
    print('Match found: ', m.group())
else:
    print('No match')

Match found:  string


In [50]:
p = re.compile("[a-z0-9_]+", re.I)
result = p.findall("SAMSUNG" "B2B samsung\nLG, SNS, Core value를 달성\
                   facebook")
print(result)

['SAMSUNGB2B', 'samsung', 'LG', 'SNS', 'Core', 'value', 'facebook']


In [51]:
segmented_words = store_segmented_words(total_df['text'])

In [52]:
# segmented_words

In [53]:
segmented_words_list = list(segmented_words)
concat_words = ""
for word in segmented_words_list:
    concat_words += word

In [54]:
# concat_words

In [55]:
result = p.findall(concat_words)
# print(result)

In [56]:
tok_words = []
untokenized_words = []
for i in result:
#     print(i)
    if len(tok(i)) > 1:
#         print(tok(i))
        tok_words.append(tok(i))
        untokenized_words.append(i)

In [57]:
data = {"tok_words":tok_words, "untokenized_word" : untokenized_words}
untokenized_eng_df = pd.DataFrame(data)

In [58]:
untokenized_eng_df

,tok_words,untokenized_word
0,"[▁B, 2, c]",B2c
1,"[▁A, G, AT, E]",AGATE
2,"[▁P, F]",PF
3,"[▁A, ur, o]",Auro
4,"[▁H, e, ad, p, h, on, e]",Headphone
...,...,...
1370,"[▁E, n, g, a, g, e, m, ent]",Engagement
1371,"[▁, AS]",AS
1372,"[▁E, S, G]",ESG
1373,"[▁, es, p, or, t, s]",esports


In [61]:
untokenized_eng_df.to_csv('data2/untokenized_eng_df.csv',encoding='utf-8-sig')

In [62]:
tok('오프라인')

['▁오', '프라', '인']